# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt", encoding='utf8',errors="ignore") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 150,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 我们的货物很紧急，你们能做到多快的船期？\n[销售回答] 我们理解您的紧急需求。我们有多条快速航线选项，并且可以提供加急服务。请告诉我具体的目的地和时间要求，我会立即为您查找最快的解决方案。')

In [6]:
len(docs)

19

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import os

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [8]:
query = "如果我的货物在运输过程中出现损坏，你们怎么处理？"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 如果我的货物在运输过程中出现损坏，你们怎么处理？
[销售回答] 我们非常重视货物安全。首先，我们会推荐您购买货物保险以降低风险。其次，如果不幸发生损坏，我们的客户服务团队将协助您进行索赔流程，确保您的损失得到及时和公正的补偿。

[客户问题] 我担心货物在长途运输中损坏，您们有何保障措施？
[销售回答] 我们提供货物保险服务和防护措施，确保货物在运输过程中得到充分保护，避免损坏或丢失的风险。

[客户问题] 您们的运输时效如何？我很关心货物能准时送达。
[销售回答] 我们有优秀的运输团队和高效的物流系统，可以确保货物按时安全送达目的地，让您放心。

[客户问题] 如果我有大量的货物需要运输，你们有没有折扣？
[销售回答] 对于大宗货物，我们确实有相应的优惠政策。请告诉我您的具体需求，我会根据货物的体积和运输频次为您计算最优惠的价格。


In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000203BDAB3A00>, search_kwargs={'k': 3})

In [ ]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

In [15]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万个舱位吗？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们如何保证舱位的及时性和可靠性？
[销售回答] 我们与全球多家船公司有着长期稳定的合作关系，通过大数据分析和市场预测，我们能够提前规划舱位，确保即使在旺季也能为客户提供所需的舱位。

[客户问题] 我们的货物很紧急，你们能做到多快的船期？
[销售回答] 我们理解您的紧急需求。我们有多条快速航线选项，并且可以提供加急服务。请告诉我具体的目的地和时间要求，我会立即为您查找最快的解决方案。

[客户问题] 为什么我应该选择你们而不是直接和船公司合作？
[销售回答] 作为专业的集装箱舱位代理，我们不仅能提供与船公司相同的服务，还能提供更加个性化和灵活的服务。此外，我们在处理突发情况和提供定制解决方案方面拥有更多经验和资源。


#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 如果我的货物在运输过程中出现损坏，你们怎么处理？
[销售回答] 我们非常重视货物安全。首先，我们会推荐您购买货物保险以降低风险。其次，如果不幸发生损坏，我们的客户服务团队将协助您进行索赔流程，确保您的损失得到及时和公正的补偿。

[客户问题] 我担心货物在长途运输中损坏，您们有何保障措施？
[销售回答] 我们提供货物保险服务和防护措施，确保货物在运输过程中得到充分保护，避免损坏或丢失的风险。

[客户问题] 您们的运输时效如何？我很关心货物能准时送达。
[销售回答] 我们有优秀的运输团队和高效的物流系统，可以确保货物按时安全送达目的地，让您放心。


### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 如果我的货物在运输过程中出现损坏，你们怎么处理？\n[销售回答] 我们非常重视货物安全。首先，我们会推荐您购买货物保险以降低风险。其次，如果不幸发生损坏，我们的客户服务团队将协助您进行索赔流程，确保您的损失得到及时和公正的补偿。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 如果我的货物在运输过程中出现损坏，你们怎么处理？\n',
 '我们非常重视货物安全。首先，我们会推荐您购买货物保险以降低风险。其次，如果不幸发生损坏，我们的客户服务团队将协助您进行索赔流程，确保您的损失得到及时和公正的补偿。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'我们非常重视货物安全。首先，我们会推荐您购买货物保险以降低风险。其次，如果不幸发生损坏，我们的客户服务团队将协助您进行索赔流程，确保您的损失得到及时和公正的补偿。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [ ]:
query = "我想要大点的舱位"

print(sales(query))

In [26]:
print(sales(query, 0.75))

['我们与全球多家船公司有着长期稳定的合作关系，通过大数据分析和市场预测，我们能够提前规划舱位，确保即使在旺季也能为客户提供所需的舱位。']


In [ ]:
query = "价格1万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "你们有没有1000万个舱位吗？"})

In [30]:
qa_chain({"query": "有什么样的舱位"})

{'query': '有什么样的舱位',
 'result': '我们提供的舱位种类可能包括但不限于标准集装箱舱位、冷藏集装箱舱位、特种集装箱舱位（例如开顶集装箱、平板集装箱等）、散货舱位、滚装船舱位等，适用于不同类型的货物运输需求。具体的舱位类型和配置，可以根据您的货物特性和运输需求来确定。如果您有特定的货物类型或特殊需求，请告知我们，以便我们为您提供更详细的信息和相应的服务。'}

In [31]:
print(sales("有什么样的舱位"))

['我们与全球多家船公司有着长期稳定的合作关系，通过大数据分析和市场预测，我们能够提前规划舱位，确保即使在旺季也能为客户提供所需的舱位。']


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
db = FAISS.load_local("real_estates_sale",OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "我想买大量舱位，你们有吗？"})

In [35]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [ ]:
qa_chain({"query": "我想买大量舱位，你们有吗？"})

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买大量舱位，你们有吗？"})

In [39]:
result

{'query': '我想买大量舱位，你们有吗？',
 'result': '对不起，我不能提供这个信息，因为我是一个人工智能，无法进行实际的交易或销售。我建议你直接联系航空公司或者旅行代理商。',
 'source_documents': []}